<a href="https://colab.research.google.com/github/omkarbhosale1623/My-Projects/blob/main/Implementing_a_RAG_system_using_Hugging_Face_transformers_and_a_retrieval_model_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install torch
!pip install langchain_community
!pip install bitsandbytes
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.2/396.2 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/1

In [6]:
from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
from time import time
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

In [7]:
model_id = 'meta-llama/Llama-2-7b-chat-hf'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

In [8]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGrained).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your termin

In [9]:
#!pip install -U bitsandbytes transformers accelerate
def load_model_and_tokenizer():
    time_1 = time()

    model_config = AutoConfig.from_pretrained(model_id)

    tokenizer = AutoTokenizer.from_pretrained(model_id)

    time_2 = time()
    print(f"Prepared model and tokenizer in {round(time_2 - time_1, 3)} seconds.")

    return AutoModelForCausalLM.from_pretrained(
        model_id,
        trust_remote_code=True,
        config=model_config,
        quantization_config=bnb_config,
        device_map='auto',
    ), tokenizer

model, tokenizer = load_model_and_tokenizer()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [10]:
time_1 = time()
query_pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",)
time_2 = time()
print(f"Prepared pipeline in {round(time_2 - time_1, 3)} seconds.")

Prepared pipeline in 12.555 seconds.


In [12]:
def test_model(tokenizer, pipeline, prompt_to_test):
  """
  Perform a query
  print the result
  Args:
    tokenizer: tokenizer
    pipeline: the pipeline
    prompt_to_test: the prompt
  Returns
    None
  """
  time_1 = time()
  sequences = pipeline(
      prompt_to_test,
      do_sample=True,
      top_k=10,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id,
      max_length=200,
  )
  time_2 = time()
  print(f"Generated text in {round(time_2 - time_1, 3)} seconds.")
  for seq in sequences:
      print(f"Result: {seq['generated_text']}")

In [13]:
test_model(tokenizer,
           query_pipeline,
           "Please explain what is the State of the Union address. Give just a defination. Keep it in 100 words.")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Generated text in 10.7 seconds.
Result: Please explain what is the State of the Union address. Give just a defination. Keep it in 100 words.

The State of the Union address is an annual speech given by the President of the United States to a joint session of Congress, typically in January of each year. In the speech, the President provides an update on the state of the union, including accomplishments, challenges, and policy proposals. The address is intended to provide a comprehensive overview of the current state of the country and to set the legislative agenda for the upcoming year.


In [15]:
llm = HuggingFacePipeline(pipeline=query_pipeline)
llm(prompt="Please what is the state of the Union address. Give just a definition. Keep it in 100 words.")

<ipython-input-15-cdbac85b2a96>:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  llm = HuggingFacePipeline(pipeline=query_pipeline)
<ipython-input-15-cdbac85b2a96>:2: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  llm(prompt="Please what is the state of the Union address. Give just a definition. Keep it in 100 words.")


"Please what is the state of the Union address. Give just a definition. Keep it in 100 words.\nThe State of the Union address is an annual speech delivered by the President of the United States to a joint session of Congress, typically in January. The address provides an update on the state of the union, including the President's legislative agenda and major policy initiatives. It is a significant event in the American political calendar and is broadcast live on television and radio."

In [16]:
loader = TextLoader("/content/biden-sotu-2023-autogenerated-transcript.txt", encoding="utf-8")
documents = loader.load()

In [17]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
all_splits = text_splitter.split_documents(documents)

In [19]:
!pip install sentence-transformers

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}

embeddings = HuggingFaceEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs
)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 5.3 MB/s eta 0:00:00


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [21]:
!pip install chromadb

vectordb = Chroma.from_documents(
    documents=all_splits,
    embedding=embeddings,
    persist_directory="chroma_db"
)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 73.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 105.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.7/149.7 kB 13.5 MB/s eta 0:00

In [22]:
retriever = vectordb.as_retriever()

qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=retriever, verbose=True)


In [23]:
def test_rag(qa, query):
  print(f"Query: {query}\n")
  time_1 = time()
  result = qa.run(query)
  time_2 = time()
  print(f"Inference time: {round(time_2-time_1, 3)} sec.")
  print(f"\nResult: {result}\n")

In [24]:
query = "What were the main topics in the State of the Union in 2023? Summarize. Keep it in 100 words."
test_rag(qa, query)

Query: What were the main topics in the State of the Union in 2023? Summarize. Keep it in 100 words.



> Entering new RetrievalQA chain...


<ipython-input-23-7485b001b624>:4: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  result = qa.run(query)



> Finished chain.
Inference time: 14.756 sec.

Result: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

of light over dark

hope over fear you never do the

stability over chaos

we have to see each other not as enemies

but as fellow Americans

we're good people the only nation in the

world built on an idea the only one

other nations

are defined by geography ethnicity

[Applause]

but we're the only nation based on an

idea

that all of us

every one of us is created equal in the

image of God

a nation that stands as a beacon of the

world

a nation

and a new age of possibilities

so I've come to

fulfill my constitutional obligation to

report in the state of the union and

here's my report

because the soul of this nation is

strong

because the backboken backbone of this

nation is strong because the people of

this nation are strong the state of the

union is st

In [26]:
docs = vectordb.similarity_search(query)
print(f"Query: {query}\n")
print(f"Retrieved documents: {len(docs)}")
for doc in docs:
  doc_details = doc.to_json()['kwargs']
  print("Source: ", doc_details['metadata']['source'])
  print("Text: ", doc_details['page_content'],"\n")

Query: What were the main topics in the State of the Union in 2023? Summarize. Keep it in 100 words.

Retrieved documents: 4
Source:  /content/biden-sotu-2023-autogenerated-transcript.txt
Text:  of light over dark

hope over fear you never do the

stability over chaos

we have to see each other not as enemies

but as fellow Americans

we're good people the only nation in the

world built on an idea the only one

other nations

are defined by geography ethnicity

[Applause]

but we're the only nation based on an

idea

that all of us

every one of us is created equal in the

image of God

a nation that stands as a beacon of the

world

a nation

and a new age of possibilities

so I've come to

fulfill my constitutional obligation to

report in the state of the union and

here's my report

because the soul of this nation is

strong

because the backboken backbone of this

nation is strong because the people of

this nation are strong the state of the

union is strong

thank you

I'm not 